In [1]:

from utils.inc_net import IncrementalNet,SimpleCosineIncrementalNet,SimpleVitNet
import foolbox
import pandas as pd
import torch
import torch.nn as nn
from torch.nn import Module
import torch.optim as optim
import numpy as np

from utils.data_manager import DataManager
from utils.toolkit import tensor2numpy, accuracy
from torch.utils.data import DataLoader
from art.attacks.evasion import AutoAttack 
from art.attacks.evasion import FastGradientMethod
import eagerpy as ep
from foolbox import PyTorchModel, accuracy, samples
from foolbox.attacks import LinfPGD, FGSM, L2CarliniWagnerAttack
from autoattack import AutoAttack
from timm.models import load_checkpoint, create_model

In [2]:
def _set_random(seed=1):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
def _set_device(args):
    device_type = args["device"]
    gpus = []

    for device in device_type:
        if device == -1:
            device = torch.device("cpu")
        else:
            device = torch.device("cuda:{}".format(device))

        gpus.append(device)

    args["device"] = gpus
def load_json(setting_path):
    import json
    with open(setting_path) as data_file:
        param = json.load(data_file)
    return param

In [3]:
from models.simplecil import Learner
json = "./exps/simplecil.json"

In [ ]:
from models.adam_adapter import Learner
json = "./exps/adam_adapter.json"

In [4]:
args = load_json(json)
args["seed"] = args["seed"][0]
#args["device"] = args["device"][0]
_set_random(args["seed"])
#_set_device(args)
args["device"][0] = "cuda:4"
torch.device(args["device"][0])

args["batch_size"] =64

In [144]:
del data_manager

In [145]:
del model, fmodel, images, labels

NameError: name 'model' is not defined

In [146]:
import gc
#torch.device(args["device"][0])
torch.cuda.empty_cache()
gc.collect()

960

In [5]:
def _compute_accuracy(model, loader):
    model.eval()
    correct, total = 0, 0
    for i, (_, inputs, targets) in enumerate(loader):
        inputs = inputs.to(model._device)
        with torch.no_grad():
            outputs = model(inputs)
        predicts = torch.max(outputs, dim=1)[1]
        correct += (predicts.cpu() == targets).sum()
        total += len(targets)
        del inputs
    return np.around(tensor2numpy(correct) * 100 / total, decimals=2)

In [ ]:
class WrapperModel(SimpleVitNet):
    def __init__(self,args):
        super().__init__(args,True)
        
        #self._cur_task += 1
        #self._total_classes = model._known_classes + data_manager.get_task_size(model._cur_task)
        self.update_fc(10)
        self.args = args
        self._device = args["device"][0]
    def __call__(self, inputs):
        return super().__call__(inputs)["logits"]
    def replace_fc(self, trainloader, train_dataset):
        model = super().eval()
        model.to(self._device)
        embedding_list = []
        label_list = []
        with torch.no_grad():
            for i, batch in enumerate(trainloader):
                (_,data, label) = batch
                data = data.to(self._device)
                label = label.to(self._device)
                embedding = model.backbone(data)
                embedding_list.append(embedding.cpu())
                label_list.append(label.cpu())
        embedding_list = torch.cat(embedding_list, dim=0)
        label_list = torch.cat(label_list, dim=0)

        class_list = np.unique(train_dataset.labels)
        proto_list = []
        for class_index in class_list:
            # print('Replacing...',class_index)
            data_index = (label_list == class_index).nonzero().squeeze(-1)
            embedding = embedding_list[data_index]
            proto = embedding.mean(0)
            self.fc.weight.data[class_index] = proto


In [6]:
class WrapperModel(Module, Learner):
    def __init__(self,args):
        Module.__init__(self)
        #super(Learner, self).__init__(args)
        Learner.__init__(self,args)
        
        #self._cur_task += 1
        #self._total_classes = model._known_classes + data_manager.get_task_size(model._cur_task)
        #self._network.update_fc(10)
        self.args = args
        self._device = args["device"][0]
    def eval(self):
        self._network.eval()
    def __call__(self, inputs):
        return self._network(inputs)["logits"]
    def forward(self, inputs):
        #return Learner.__call__(x)["logits"]
        return self._network(inputs)["logits"]
        


In [128]:
args["batch_size"]

64

In [8]:
batch_size = args["batch_size"]

num_workers = 1

In [9]:
_total_classes = 10
data_manager = DataManager(
        args["dataset"],
        args["shuffle"],
        args["seed"],
        args["init_cls"],
        args["increment"],
        args,
    )
train_dataset = data_manager.get_dataset(np.arange(0, _total_classes),source="train", mode="train", )

model = WrapperModel(args)
model.train_dataset=train_dataset
_total_classes = 0 + data_manager.get_task_size(0)

train_dataset_for_protonet = data_manager.get_dataset(np.arange(0, _total_classes),source="train", mode="test", )
train_loader_for_protonet = DataLoader(train_dataset_for_protonet, batch_size=batch_size, shuffle=True, num_workers=num_workers)

model._network.to(model._device)
model._network.update_fc(_total_classes)
model.replace_fc(train_loader_for_protonet, model._network, args)

#model._network.update_fc(_total_classes)

Files already downloaded and verified
Files already downloaded and verified
This is for the BaseNet initialization.
After BaseNet initialization.


SimpleVitNet(
  (backbone): VisionTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      (norm): Identity()
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (patch_drop): Identity()
    (norm_pre): Identity()
    (blocks): Sequential(
      (0): Block(
        (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=768, out_features=2304, bias=True)
          (q_norm): Identity()
          (k_norm): Identity()
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=768, out_features=768, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (ls1): Identity()
        (drop_path1): Identity()
        (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (act): GELU(approximate='non

In [10]:

#model.to(args["device"][0])
model.eval()
model._network.eval()
preprocessing = dict(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], axis=-3)
fmodel = foolbox.models.PyTorchModel(model, bounds=(0,1), device=args["device"][0], preprocessing=preprocessing)
#fmodel.to(args["device"][0])

/home/han22002/miniconda3/lib/python3.11/site-packages/foolbox/models/pytorch.py:36: UserWarning: The PyTorch model is in training mode and therefore might not be deterministic. Call the eval() method to set it in evaluation mode if this is not intended.
  warnings.warn(


In [11]:
test_dataset = data_manager.get_dataset(np.arange(0, _total_classes), source="test", mode="test" )
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)
#model.to(args["device"][0])
print(_compute_accuracy(model, test_loader))

93.8


In [12]:
acc = []
for i, batch in enumerate(test_loader):
    (_,data, label) = batch
    
    images = data.to(args["device"][0])
    labels = label.to(args["device"][0])
    acc.append(accuracy(fmodel, images, labels))
    del images, labels

In [13]:
sum(acc)/len(acc)

0.8953124992549419

In [14]:
epsilons = [0.0,
        0.001,
        0.003,
        0.005,
        0.008,
        0.01,
        1.0
           ]
steps = [1, 5, 10]#,30, 40, 50]

In [16]:
robacc_per_step = []
attack_epochs = 10
for step in steps:
    print(f"Step {step}")
    attack = LinfPGD(steps=step)
    
    clean_acc = 0.0
    robust_acc = []
    for i, batch in enumerate(test_loader):
        (_,data, label) = batch
        images = data.to(args["device"][0])
        labels = label.to(args["device"][0])
        
        # Samples (attack_batch_size * attack_epochs) images for adversarial attack.
        if i >= attack_epochs:
            break
    
        #images, labels = data[0].to(device), data[1].to(device)
        #if step == steps[0]:
        #    clean_acc += (get_acc(fmodel, images, labels)) / args.attack_epochs  # accumulate for attack epochs.
    
        
        images, labels = ep.astensors(images, labels)
        raw_advs, clipped_advs, success = attack(fmodel, images, labels, epsilons=epsilons)
    
        robust_accuracy = 1 - success.float32().mean(axis=-1)
        #print(robust_accuracy)
        robust_acc.append(robust_accuracy)# / attack_epochs
    
        #for eps, acc in zip(epsilons, robust_acc):
        #    print(f"  Step {step}, Linf norm ≤ {eps:<6}: {acc.item() * 100:4.1f} %")
        #print('  -------------------')
        del images, labels
    racc_step = 0
    for i in range(len(robust_acc)):
        racc_step += robust_acc[i].numpy()
    racc_step = racc_step/len(robust_acc)
    robacc_per_step.append(racc_step)

Step 1


KeyboardInterrupt: 